# Stock data analysis

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [8]:
display_analysis("GOOG")

### Analysis and Summary of GOOG Stock Trends

The provided data represents the weekly closing prices and key metrics for Google (GOOG) over a span of time. Here’s a breakdown of the observed trends and a calculation of the total returns over this period.

#### Key Metrics Explained
1. **Open Price**: The price of the stock when the market opens for that week.
2. **High Price**: The maximum price the stock reached during the week.
3. **Low Price**: The minimum price the stock reached during the week.
4. **Close Price**: The price of the stock when the market closes for that week.
5. **Adjusted Close Price**: The closing price adjusted for stock splits and dividends, giving a better representation of the investment's value.
6. **Volume**: The number of shares traded during that week.

#### Observed Trends
- **Peak Performance**: The highest closing value in the analyzed period was **205.60** on January 27, 2025, showing strong recent performance.
- **Declining Prices**: The stock reached a lower price of **147.74** on March 31, 2025. This represents a notable drop from the previous high, indicating volatility and perhaps market correction.
- **Recent Recovery**: Following the drop, there was a slight recovery with the stock closing at **153.90** on April 22, 2025. This suggests investors may have regained confidence.
- **Volume Fluctuations**: There’s significant variation in trading volumes, with the highest volume recorded at **187,459,400** shares traded on April 7, 2025, which shows heightened activity likely associated with price fluctuations.

#### Total Returns Calculation
To assess total returns over the period:
1. Determine the initial investment amount. Assuming starting on **April 22, 2024**, with a closing price of **173.69**.
2. Determine the final value on **April 22, 2025**, with a closing price of **153.90**.

The formula to calculate total returns is:

\[
\text{Total Returns (\%)} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100
\]

Substituting the values:

- Initial Price (April 22, 2024): **173.69**
- Final Price (April 22, 2025): **153.90**

\[
\text{Total Returns (\%)} = \left( \frac{153.90 - 173.69}{173.69} \right) \times 100 \approx -11.43\%
\]

### Conclusion
Overall, Google’s stock has seen considerable volatility over the past year with a noticeable drop in price towards the end of the analyzed period. Investors who began their positions a year ago would have experienced a total return of approximately **-11.43%**, indicating a loss. This trend highlights the importance of monitoring stock performance and the associated risks when investing.

In [9]:
display_analysis("PFE")

### Stock Analysis for PFE (Pfizer Inc.)

#### Overview of Stock Trends

Based on the provided data for Pfizer (PFE) over the timeframe of approximately one year, we can summarize several key trends:

1. **Price Movements**:
   - The stock price fluctuated significantly throughout the year, with highs and lows indicating volatility.
   - The highest recorded closing price was approximately **$29.22** in October 2024, while the lowest was around **$21.91** in early April 2025.
   - Notably, as the year progressed towards early 2025, there was a general downward trend in the stock price from September 2024 onward.

2. **Volume of Transactions**:
   - Trading volume varied considerably, with some weeks experiencing exceptionally high trading volumes (e.g., over **493 million** shares on March 17, 2025). 
   - Higher trading volumes can often indicate increased investor interest, whether due to positive or negative news affecting the company or industry.

3. **Dividends**:
   - There were periodic dividend payments noted. For instance, dividends of **$0.42** and **$0.43** were declared in parts of November and January, suggesting that Pfizer continues to return value to its shareholders.

4. **Adjusted Close Prices**:
   - The adjusted close price reflects the stock value adjusted for any dividends and stock splits. It provides a more accurate representation of an investor's actual return and is useful for assessing long-term performance.

#### Summary of Price Movement

- **Start of Period (Apr 22, 2024)**: $25.40
- **End of Period (Apr 22, 2025)**: $22.53

From this data, we can calculate the **total return** for an investor who held onto the stock from the start to the end of this period using the formula:

\[
\text{Total Return (\%)} = \left( \frac{\text{End Price} - \text{Start Price}}{\text{Start Price}} \right) \times 100
\]

Plugging in the values:

- Start Price = 25.40
- End Price = 22.53

\[
\text{Total Return (\%)} = \left( \frac{22.53 - 25.40}{25.40} \right) \times 100 \approx -11.3\%
\]

### Conclusion

1. **Overall Trend**: The overall trend indicates a **decrease** in the stock price of approximately **11.3%**, signaling a declining performance of Pfizer over the past year.
2. **Investment Caution**: Potential investors should consider the recent downward trend along with the volatility of the stock price. It's also essential to evaluate other fundamental factors affecting the stock before making an investment.
3. **Dividends**: Despite the price drop, the regularity of dividends may still present an attractive feature for income-focused investors. 

Investors should continue to monitor developments within the company that could influence stock performance, including news related to product developments, market conditions, and broader economic factors.

In [ ]:
display_analysis("AAPL")